In [1]:
import sqlite3
import pandas as pd

In [6]:
min_words = 3

conn = sqlite3.connect("../data/twitter.db")
cursor = conn.cursor()

In [3]:
data = cursor.execute("SELECT * from tweets_raw;")
raw_tweets = cursor.fetchall()
#author_id = result[0][0]

clean_df = pd.DataFrame(
            [
                (tweet[0], tweet[1], tweet[2])
                for tweet in raw_tweets
            ],
            columns=["id", "author", "text"],
        )

In [4]:
clean_df.text

0       #Python Pop Quiz 🐍❓\n\nHere we have a dataclas...
1       @mousevspython Thanks for checking out my mini...
2       You can learn more about how the `ChainMap` in...
3       Here's another example where you create a Chai...
4       Let's look at an example of using `ChainMap` i...
                              ...                        
3706    I turned my silly 3-liner #Python script to co...
3707    The PyDev of the Week is Luiz Gustavo Martins ...
3708    One of my favorite #Python modules is `shutil`...
3709    Discover how easy it is to download a file in ...
3710    What is your favorite #Python editor for begin...
Name: text, Length: 3711, dtype: object

In [10]:
clean_df.loc[0].text.replace("\n", ". ")

"#Python Pop Quiz 🐍❓\n\nHere we have a dataclass where I want to have a default list. But it doesn't work! Can you fix the code so it works? https://t.co/wmWKeAlVjh"

In [12]:
clean_df

,id,author,text
0,1542138596387438593,19067150,#Python Pop Quiz 🐍❓\n\nHere we have a dataclas...
1,1542071124737024002,19067150,@mousevspython Thanks for checking out my mini...
2,1542071120588939264,19067150,You can learn more about how the `ChainMap` in...
3,1542071115928977409,19067150,Here's another example where you create a Chai...
4,1542071111474634754,19067150,Let's look at an example of using `ChainMap` i...
...,...,...,...
3706,1542309718646996993,19067150,I turned my silly 3-liner #Python script to co...
3707,1542246549345927168,19067150,The PyDev of the Week is Luiz Gustavo Martins ...
3708,1542241312476471297,19067150,One of my favorite #Python modules is `shutil`...
3709,1542189544086306819,19067150,Discover how easy it is to download a file in ...


In [97]:
repl = lambda s: re.sub("\n+", " ", s.group(0))

(clean_df
 .assign(
     text = clean_df.text.replace("#\\w*|@\\w*|https?://t.co\\w*", "", regex=True)
                         .str.replace("[.,!?:;]\n+", repl, regex=True)
                         .replace("\n+", ". ", regex=True)
                         .replace("[`\'\"]", "", regex=True)
                         .apply(lambda x: x.strip()),
     n_words = lambda df: df.text.str.split().map(len))
 [lambda x: x.n_words >= min_words]
 .drop("n_words", axis=1)
 .reset_index(drop=True)
)


,id,author,text
0,1542138596387438593,19067150,Pop Quiz 🐍❓. Here we have a dataclass where I ...
1,1542071124737024002,19067150,Thanks for checking out my mini-thread on coll...
2,1542071120588939264,19067150,You can learn more about how the ChainMap in m...
3,1542071115928977409,19067150,Heres another example where you create a Chain...
4,1542071111474634754,19067150,Lets look at an example of using ChainMap in ...
...,...,...,...
3356,1542309718646996993,19067150,I turned my silly 3-liner script to convert a...
3357,1542246549345927168,19067150,The PyDev of the Week is Luiz Gustavo Martins ...
3358,1542241312476471297,19067150,One of my favorite modules is shutil 🐍🔥. It l...
3359,1542189544086306819,19067150,Discover how easy it is to download a file in ...


In [88]:
repl = lambda s: re.sub("\n+", " ", s.group(0))

s = pd.Series("@mousevspython Thanks for checking out my mini-thread on `collections.ChainMap`!\n\nPython is an amazing language. Follow me to learn more! 🐍🔥")
s.replace("#\w*|@\w*|https?://t.co/\w*", "", regex=True).str.replace("[.,!?:;]\n+", repl, regex=True).replace("\n+", ". ", regex=True)

0     Thanks for checking out my mini-thread on `co...
dtype: object

In [60]:
import re



s = "test.\n\nI want to clean this text\n\nFor the good of mankind!"
re.sub(r"[.,!?:;]\n+", repl, s)

'test. I want to clean this text\n\nFor the good of mankind!'